# Analiza Wielowymiarowa - Współzależność zjawisk - Zajecia 3

In [ ]:
import os
os.getcwd() # oczekiwany .../AWXXXX/materialy/zajecia03
# mozna uzyc os.chdir("path") do zmiany

In [ ]:
import yaml
spec =  yaml.safe_load(open('../../spec.yaml'))

In [ ]:
# STATA
import stata_setup
stata_setup.config(spec["stata_path"], spec["stata_type"])
from pystata import stata

In [ ]:
import pandas as pd
import scipy.stats as stats
import numpy as np
import scipy
import sklearn
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
%stata sysuse auto

In [ ]:
auto = stata.pdataframe_from_data(missingval = np.NaN, valuelabel = True)

Analiza korelacji Test chi^2  
chi2 - chi-kwadrat Pearsona dla hipotezy, że wiersze i kolumny w tabeli dwudzielnej są niezależne.
V - oznacza statystykę V Cramera.

**Wiecej podczas zajec nr 5**

In [ ]:
# %stata help tabulate twoway

Ważne aby przeanalizować liczbę i strukturę brakujacych obserwacji.
Obserwacje brakujace moga mieć istony wplyw na wyniki analizy.

In [ ]:
auto[auto.isnull().any(axis = 1)]

In [ ]:
%stata tabulate rep78 foreign, miss

In [ ]:
table_raw = pd.crosstab(auto.rep78.fillna('missing'), auto.foreign)
table_full = pd.crosstab(auto.rep78.fillna('missing'), auto.foreign, margins = True)
table_full

In [ ]:
%%stata 
tabulate rep78 foreign, missing chi2
/* Test V-Cramera */
tabulate rep78 foreign, missing V
/* Wszystkie testy zaleznosci */
tabulate rep78 foreign, all

In [ ]:
# https://stackoverflow.com/questions/33303314/confusing-behaviour-of-pandas-crosstab-function-with-dataframe-containing-nan
# link z wyjasnieniem jak stats.chi2_contingency moze zachowywac sie kiedy analizujemy dane z wartosciami brakuajcymi
stats.chi2_contingency(table_raw.values)

In [ ]:
#Chi-squared test statistic
X2 = stats.chi2_contingency(table_raw.values, correction=False)[0]
n = np.sum(table_raw.values) # sample size
minDim = min(table_raw.values.shape) - 1 # minimum of rows and columns
V = np.sqrt((X2/n) / minDim)
print(V)

### Korelacja

In [ ]:
%%stata
/* Korelacja miedzy zmiennymi ciaglymi, wspolczynnik Pearsona */
corr price mpg
/* Oczywscie mozna tez obliczyc korelacje miedzy zmienna ciagla a dyskretna, ale jest to slabo uzasadnione */
corr price mpg rep78 foreign

In [ ]:
%%stata
/* Dla zmiennych 0-1 bedacych reprezentacja zmiennych ukrytych o rozkladzie normalnym lepsza miara jest korelacja tetrachoryczna */
/* Tworzymy zmienna 0-1 o wartosci 1 dla ceny powyzej 10.000$ */
gen price_10 = (price > 10000)
/* Korelacja terachoryczna -- dla zmiennych 0-1 */
corr price_10 foreign
tetrachoric price_10 foreign

In [ ]:
auto[["price", "mpg"]].corr()

In [ ]:
# zmienna typu string na numeryczna
auto["foreign_b"] = auto["foreign"].astype("category").cat.codes

In [ ]:
auto[["price", "mpg", "rep78", "foreign_b"]].corr()

In [ ]:
# cos innego tez dla binarnych - point biserial correlation
stats.pointbiserialr(auto.price > 10000, auto.foreign_b)

In [ ]:
%%stata
/* Wspolczynnik korelacji Kendalla */
ktau price mpg rep78 foreign
/* Parametr star definiuje przy jakiej wartosci p program wyswietla znak "*" oznaczajacy brak podstaw do odrzucenia hipotezy o braku korelacji */
ktau price mpg rep78 foreign, star(0.05)
/* Opcja stats(p) pozwala wyswietlic wartosc p dla kazdego wspolczynnika */
ktau price mpg rep78 foreign, star(0.05) stats(p)

In [ ]:
auto[["price", "mpg", "rep78", "foreign_b"]].corr(method = "kendall")

In [ ]:
%%stata
/* Porownanie wartosci wspolczynnikow Pearsona i Kendala (tau-b) */
/* Dwie zmienne 0-1 */
corr price_10 foreign
ktau price_10 foreign
/* Zmienna 0-1 i ciagla */
corr price foreign
ktau price foreign
/* Dwie zmienne ciagle */
corr price gear_ratio
ktau price gear_ratio

In [ ]:
%%stata
/* Wspolczynnik korelacji Spearmana */
spearman price mpg rep78 foreign
/* Parametr star definuje przy jakiej wartosci p program wyswietla znak "*" oznaczajacy brak podstaw do odrzucenia hipotezy o braku korelacji */
spearman price mpg rep78 foreign, star(0.05)

In [ ]:
auto[["price", "mpg", "rep78", "foreign_b"]].corr(method = "spearman")

In [ ]:
%%stata
/* Porownanie wartosci wspolczynnikow Pearsona i Spearmana */
/* Dwie zmienne 0-1 */
corr price_10 foreign
spearman price_10 foreign
/* Zmienna 0-1 i ciagla */
corr price foreign
spearman price foreign
/* Dwie zmienne ciagle */
corr price gear_ratio
spearman price gear_ratio

### Analiza zróżnicowania

In [ ]:
#%stata help anova

In [ ]:
%%stata
/* Jednoczynnikowa analiza wariancji */
anova price foreign
/* I dla porownania regresja */
reg price foreign

In [ ]:
model = ols('price ~ C(foreign)', data=auto).fit()
anova_table = sm.stats.anova_lm(model)
anova_table

In [ ]:
model.summary()

In [ ]:
%%stata
/* Anova traktuje kazda zmienna "objasniajaca" jako dyskretna */
anova price mpg
/* opcerator c. sprawia ze zmienna traktowana jest jako zmienna o rozkladzie ciaglym */
anova price c.mpg

In [ ]:
model = ols('price ~ C(mpg)', data=auto).fit()
anova_table = sm.stats.anova_lm(model)
anova_table

In [ ]:
model = ols('price ~ mpg', data=auto).fit()
anova_table = sm.stats.anova_lm(model)
anova_table

### Wieloczynnikowa analiza wariancji

Założeniem ANOVA jest niezależność zmiennych tłumaczących wariancję. Taki model w pracy Herr (1986) On the History of ANOVA
in Unbalanced, Factorial Designs: The First 30 Years, 
The American Statistician, Nov., 1986, Vol. 40, No. 4 (Nov., 1986), pp. 265-270 jest określany jako ANOVA typu 1.

In [ ]:
# Nie jest latwo użyć ANOVA typu 1 w pakiecie statsmodels 
# https://github.com/statsmodels/statsmodels/issues/1112
# Dodatkowo wydaje się też ze to druga zmienna nie pierwsza jest brana pod uwage bez warunkowania

In [ ]:
%%stata
anova price foreign c.mpg

In [ ]:
model = ols('price ~ foreign_b + mpg', data=auto).fit()
anova_table = sm.stats.anova_lm(model, typ = 2)
anova_table

Model ANOVA z interakcjami jest nazywany ANOVA typu 2. 

In [ ]:
model = ols('price ~ mpg + foreign_b', data=auto).fit()
anova_table = sm.stats.anova_lm(model, typ = 2)
anova_table

In [ ]:
%%stata
/* "#" pozwala na proste uwzglednienie interakcji */
anova price foreign c.mpg foreign#c.mpg

In [ ]:
model = ols('price ~ foreign_b + mpg + foreign_b:mpg', data=auto).fit()
anova_table = sm.stats.anova_lm(model, typ = 1)
anova_table

In [ ]:
%stata clear

### Ćwiczenie
Bedziemy pracować z danymi pochodzącymi z badania Diagnoza Spoleczna z 2005 roku  
Proszę ściagnąć dane ze strony www.ekonometria.wne.uw.edu.pl i załadować zbiór osób od pakietu statystycznego np. [Stata] 
Proszę zapoznać się z zawartoscią zbioru i odpowiedzieć na pytania, przeprowadzająć odpowiednie testy

- Czy waga jest zwiazana z plcia?  
- Czy wzrost jest zwiazany z plcia?  
- Czy wyksztalcenie jest powiazane z plcia? 
- Czy wsrod osob o wyzszym wyksztalceniu waga respondenta jest zwiazana z plcia? 
- Czy wsrod osob o wyzszym wyksztalceniu wzrost respondenta jest zwiazany z plcia?
- Czy wsrod osob o wyzszym wyksztalceniu deklarowane zarobki sa zwiazane z plcia?
